In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("../Ntu_DataSet/Orders.csv")

c:\users\alex\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df2=pd.read_csv("../Ntu_DataSet/PromotionOrders.csv")

In [4]:
promotion_data=df2.copy()

In [18]:
shopping_data=df.copy()

In [6]:
shopping_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5495276 entries, 0 to 5495275
Data columns (total 28 columns):
DateId                         int64
MemberId                       int64
OrderGroupCode                 object
TrackSourceTypeDef             object
TrackDeviceTypeDef             object
PayProfileTypeDef              object
SalesOrderSlaveId              int64
SalePageId                     int64
IsMajor                        bool
IsGift                         bool
IsSalePageGift                 bool
Quantity                       int64
UnitPrice                      float64
PromotionDiscount              float64
ECouponId                      int64
ECouponDiscount                float64
SalesOrderSlaveTotalPayment    float64
SalesOrderSlaveDateTime        object
SalesOrderReceiverId           int64
City                           object
District                       object
ZipCode                        float64
StoreName                      object
StoreId              

In [19]:
shopping_data=shopping_data.ix[:,["DateId","MemberId","OrderGroupCode","IsGift","IsSalePageGift","Quantity","UnitPrice","PromotionDiscount","EcouponId","ECouponDiscount","SalesOrderSlaveTotalPayment"]]

c:\users\alex\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [20]:
shopping_data=shopping_data.fillna(0)

In [22]:
shopping_data['DateId']=pd.to_datetime(shopping_data['DateId'],format="%Y%m%d") 

以3個月為單位，故分成  2015/07~2015/09，2015/10~2016/12，2016/01~2016/03，2016/04~2016/06，2016/07~2016/09，2016/10~2017/12，2017/01~2017/03  七個時段

In [24]:
shoppingDataList=[]
timeExpend=["2015-07~2015-09","2015-10~2016-12","2016-01~2016-03","2016-04~2016-06","2016-07~2016-09","2016-10~2017-12","2017-01~2017-03"]
for time in timeExpend:
    time=time.split("~")
    temp=shopping_data[shopping_data["DateId"]>=time[0]]
    temp=temp[temp["DateId"]<=time[1]]
    shoppingDataList.append(temp)

In [58]:
shoppingDataList[3]

,DateId,MemberId,OrderGroupCode,IsGift,IsSalePageGift,Quantity,UnitPrice,PromotionDiscount,EcouponId,ECouponDiscount,SalesOrderSlaveTotalPayment
1555460,2016-04-02,1874012,"b""'\xadY\x0e\xab\xbcp\xd7D\xb2O\xb1T\xbe\xec\x...",False,False,1,249.0,0.0,0.0,0.0,249.0
1555461,2016-04-02,1300360,b'\x93\x81o\xe8\x8e\xca\xc6\xdb\x9az\x1b\xf4\x...,False,False,1,199.0,0.0,0.0,0.0,199.0
1555462,2016-04-02,1300360,b'\x93\x81o\xe8\x8e\xca\xc6\xdb\x9az\x1b\xf4\x...,False,False,1,199.0,0.0,0.0,0.0,199.0
1555463,2016-04-02,1229126,b'3\xf8\x97\xcd\xbd\x9d+\\\xe5\xa5b\x18\xb4.\x...,False,False,1,199.0,0.0,0.0,-58.0,141.0
1555464,2016-04-02,1229126,b'3\xf8\x97\xcd\xbd\x9d+\\\xe5\xa5b\x18\xb4.\x...,False,False,1,149.0,0.0,0.0,-42.0,107.0
1555465,2016-04-02,540366,b'i\xbf\xa9-\x96\xc3\xc0\xad\x97\xa8_q\x9bx\x9...,False,False,1,229.0,0.0,0.0,0.0,229.0
1555466,2016-04-02,540366,b'i\xbf\xa9-\x96\xc3\xc0\xad\x97\xa8_q\x9bx\x9...,False,False,1,199.0,0.0,0.0,0.0,199.0
1555467,2016-04-02,3053541,b'\xeaw\x82/!Q\xc6\xa7xX\x87\xe3Y\x9f\x9by\x00...,False,False,1,199.0,0.0,0.0,-28.0,171.0
1555468,2016-04-02,3053541,b'\xeaw\x82/!Q\xc6\xa7xX\x87\xe3Y\x9f\x9by\x00...,False,False,1,199.0,0.0,0.0,-26.0,173.0
1555469,2016-04-02,3053541,b'\xeaw\x82/!Q\xc6\xa7xX\x87\xe3Y\x9f\x9by\x00...,False,False,1,350.0,0.0,0.0,-46.0,304.0


In [85]:
newCustomer=[]
activeCustomer=[]
sleepCustomer=[]
customerTotal=[]
customers=[]
for index,customer in enumerate(shoppingDataList):
    if index==0:
        newCustomer.append(set(customer["MemberId"]))
        customerTotal.append(set(customer["MemberId"]))
        activeCustomer.append({})
        sleepCustomer.append({})
    else:
#         customerTotal------->所有的消費者
        customerTotalItem=customerTotal[index-1] | set(tempData["MemberId"])
        customerTotal.append(customerTotalItem)
#         newCustomer--------->新增加的消費者且期間銷售次數為一次
        tempData=customer.ix[:,["MemberId","OrderGroupCode"]]
        tempGroup=tempData.groupby("MemberId")["OrderGroupCode"].nunique()
        newCustomerItem=set(tempGroup.index[tempGroup.values==1])
        newCustomer.append(newCustomerItem)
#         activeCustomer------>主要消費者
        activeCustomerItem=set(tempData["MemberId"])-newCustomerItem
        activeCustomer.append(activeCustomerItem)
#         sleepCustomer------->上一期有消費但這期沒消費
        sleepCustomerItem=customerTotal[index-1] - set(tempData["MemberId"])
        sleepCustomer.append(sleepCustomerItem)

In [86]:
for i in range(len(newCustomer)):
    print("NewCustomer:",len(newCustomer[i]))
    print("ActiveCustomer:",len(activeCustomer[i]))
    print("SleepCustomer:",len(sleepCustomer[i]))
    print("Customers:",len(customers[i]))
    print("TotalCustomer:",len(customerTotal[i]))
    print("---------------")

NewCustomer: 23748
ActiveCustomer: 0
SleepCustomer: 0
Customers: 23748
TotalCustomer: 23748
---------------
NewCustomer: 197630
ActiveCustomer: 124314
SleepCustomer: 13648
Customers: 321944
TotalCustomer: 335592
---------------
NewCustomer: 27866
ActiveCustomer: 6257
SleepCustomer: 301469
Customers: 34123
TotalCustomer: 335592
---------------
NewCustomer: 60474
ActiveCustomer: 17196
SleepCustomer: 257922
Customers: 77670
TotalCustomer: 335592
---------------
NewCustomer: 70611
ActiveCustomer: 17750
SleepCustomer: 247231
Customers: 88361
TotalCustomer: 335592
---------------
NewCustomer: 186864
ActiveCustomer: 123916
SleepCustomer: 192608
Customers: 310780
TotalCustomer: 503388
---------------
NewCustomer: 29353
ActiveCustomer: 6326
SleepCustomer: 467709
Customers: 35679
TotalCustomer: 503388
---------------


In [87]:
a={1,2,5,87,4}
b={1,5,1,62,4}
a | b

{1, 2, 4, 5, 62, 87}

In [56]:
shoppingDataList[0]=shoppingDataList[0].ix[:,["MemberId","OrderGroupCode"]]

a=shoppingDataList[0].groupby("MemberId")["OrderGroupCode"].nunique()
a.index[a.values==1]

c:\users\alex\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


Int64Index([   1373,    2723,    2940,    3270,    3921,    4077,    4652,
               4805,    5441,    5541,
            ...
            1683433, 1683480, 1683571, 1683628, 1683656, 1683915, 1683917,
            1683958, 1683984, 6486287],
           dtype='int64', name='MemberId', length=19538)

1218